show config

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

node configs

In [ ]:
slice_name="router_aggregation_advanced"

node_conf = [
    {'name': "router1",  'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_20', 'packages': ['net-tools']}, 
    {'name': "client1",   'cores': 1, 'ram': 4, 'disk': 6, 'image': 'default_ubuntu_20', 'packages': ['net-tools']}, 
    {'name': "client2",   'cores': 1, 'ram': 4, 'disk': 6, 'image': 'default_ubuntu_20', 'packages': ['net-tools']}, 
    
    {'name': "router2",  'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_20', 'packages': ['net-tools']},
    {'name': "client3",   'cores': 1, 'ram': 4, 'disk': 6, 'image': 'default_ubuntu_20', 'packages': ['net-tools']}, 
    {'name': "client4",   'cores': 1, 'ram': 4, 'disk': 6, 'image': 'default_ubuntu_20', 'packages': ['net-tools']}, 
    
    {'name': "hooke",  'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_20', 'packages': ['net-tools']}, 
    {'name': "newton",  'cores': 2, 'ram': 4, 'disk': 10, 'image': 'default_ubuntu_20', 'packages': ['net-tools']}, 
]
net_conf = [
    {"name": "net_r1", "subnet": "10.10.1.0/24", 
      "nodes": [{"name": "client1",  "addr": "10.10.1.100"}, {"name": "client2", "addr": "10.10.1.200"}, {"name": "router1", "addr": "10.10.1.10"}]},
  
    {"name": "net_r2", "subnet": "10.10.2.0/24", 
      "nodes": [{"name": "client3",  "addr": "10.10.2.100"}, {"name": "client4",  "addr": "10.10.2.200"}, {"name": "router2", "addr": "10.10.2.10"}]},
 
    {"name": "net_newton", "subnet": "10.10.3.0/24", 
      "nodes": [{"name": "newton",  "addr": "10.10.3.100"}, {"name": "router1", "addr": "10.10.3.10"}, {"name": "router2", "addr": "10.10.3.20"}]},
    {"name": "net_hooke", "subnet": "10.10.4.0/24", 
      "nodes": [{"name": "hooke",  "addr": "10.10.4.100"}, {"name": "router1", "addr": "10.10.4.10"}, {"name": "router2", "addr": "10.10.4.20"}]},
  
#  {"name": "net_r", "subnet": "10.10.1.0/24", "nodes": [{"name": "romeo",  "addr": "10.10.1.100"}, {"name": "router", "addr": "10.10.1.10"}]},
#  {"name": "net_j", "subnet": "10.10.2.0/24", "nodes": [{"name": "juliet", "addr": "10.10.2.100"}, {"name": "router", "addr": "10.10.2.10"}]},
]
route_conf = [
 {"addr": "10.10.3.0/24", "gw": "10.10.1.10", "nodes": ["client1","client2"]}, 
 {"addr": "10.10.4.0/24", "gw": "10.10.1.10", "nodes": ["client1","client2"]}, 
 {"addr": "10.10.3.0/24", "gw": "10.10.2.10", "nodes": ["client3","client4"]}, 
 {"addr": "10.10.4.0/24", "gw": "10.10.2.10", "nodes": ["client3","client4"]}, 
 
 {"addr": "10.10.1.0/24", "gw": "10.10.3.10", "nodes": ["newton"]}, 
 {"addr": "10.10.2.0/24", "gw": "10.10.3.20", "nodes": ["newton"]}, 
 
 {"addr": "10.10.1.0/24", "gw": "10.10.4.10", "nodes": ["hooke"]}, 
 {"addr": "10.10.2.0/24", "gw": "10.10.4.20", "nodes": ["hooke"]}, 
 
#  {"addr": "10.10.1.0/24", "gw": "10.10.2.10", "nodes": ["juliet"]}, 
#  {"addr": "10.10.2.0/24", "gw": "10.10.1.10", "nodes": ["romeo"]}
]
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]) }

Reserve resources

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

In [ ]:
# select a random site
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

# fablib.show_site(site_name)
print(site_name)

In [ ]:
# this cell sets up the nodes
for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                cores=n['cores'], 
                ram=n['ram'], 
                disk=n['disk'], 
                image=n['image'])

In [ ]:
# this cell sets up the network segments
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

In [ ]:
slice.submit()

In [ ]:
# install packages
# this will take a while and will run in background while you do other steps
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

In [ ]:
# bring interfaces up and either assign an address (if there is one) or flush address
from ipaddress import ip_address, IPv4Address, IPv4Network

for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

In [ ]:
# prepare a "hosts" file that has names and addresses of every node
hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]
for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

In [ ]:
# enable IPv4 forwarding on all nodes
for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

In [ ]:
# set up static routes
for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
# extend expiration
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end date to 7 days from now
end_date = (datetime.now(timezone.utc) + timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
slice.renew(end_date)